In [402]:
import numpy as np
import matplotlib as plt
from typing import Optional, List

# Установим seed для воспроизводимости
np.random.seed(42)


In [403]:
class Layer:
    """
    Базовый класс для всех слоев нейронной сети
    """

    def __init__(self):
        self.training = True

    def forward(self, x):
        """
        Прямое распространение
        """
        raise NotImplementedError

    def backward(self, grad_output):
        """
        Обратное распространение
        """
        raise NotImplementedError

    def train(self):
        """
        Переключение в режим обучения
        """
        self.training = True

    def eval(self):
        """
        Переключение в режим инференса
        """
        self.training = False

    def __call__(self, x):
        return self.forward(x)


In [404]:
class ReLU(Layer):
    def __init__(self):
        super().__init__()
        self.input = None

    def forward(self, x):
        """
        Прямое распространение для ReLU
        
        Args:

        
        Returns:
            выходной тензор той же формы
        """
        # TODO: Сохраните входные данные для backward pass
        self.input = x

        # TODO: Реализуйте ReLU функцию
        output = np.maximum(0, x)
        return output

    def backward(self, grad_output):
        """
        Обратное распространение для ReLU
        
        Args:
            grad_output: градиент от следующего слоя
        
        Returns:
            градиент для предыдущего слоя
        """
        # TODO: Реализуйте градиент ReLU
        self.mask = self.input > 0
        grad_input = grad_output * self.mask

        return grad_input


In [405]:
class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
        self.output = None

    def forward(self, x):
        """
        Прямое распространение для Sigmoid
        
        Args:
            x: входной тензор
        
        Returns:
            выходной тензор той же формы, значения в диапазоне (0, 1)
        """
        # TODO: Реализуйте sigmoid функцию
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, grad_output):
        """
        Обратное распространение для Sigmoid
        
        Args:
            grad_output: градиент от следующего слоя
        
        Returns:
            градиент для предыдущего слоя
        """
        # TODO: Реализуйте градиент sigmoid
        derivative_sigm = self.output * (1 - self.output)
        grad_input = derivative_sigm * grad_output
        return grad_input


In [406]:
class Tanh(Layer):
    def __init__(self):
        super().__init__()
        self.output = None

    def forward(self, x):
        """
        Прямое распространение для Tanh
        
        Args:
            x: входной тензор
        
        Returns:
            выходной тензор той же формы, значения в диапазоне (-1, 1)
        """
        # TODO: Реализуйте tanh функцию
        self.output = np.tanh(x)
        return self.output

    def backward(self, grad_output):
        """
        Обратное распространение для Tanh
        
        Args:
            grad_output: градиент от следующего слоя
        
        Returns:
            градиент для предыдущего слоя
        """
        # TODO: Реализуйте градиент tanh

        derivative_tanh = 1 - self.output ** 2
        grad_input = grad_output * derivative_tanh

        return grad_input


In [407]:
class Linear(Layer):
    def __init__(self, input_size, output_size, bias=True):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.use_bias = bias

        # TODO: Инициализируйте веса
        limit = np.sqrt(6 / (input_size + output_size))
        self.weight = np.random.uniform(-limit, limit, (input_size, output_size))

        # TODO: Инициализируйте bias (если используется)
        if self.use_bias:
            self.bias = np.zeros(output_size)
        else:
            self.bias = None

        # Переменные для сохранения входных данных и градиентов
        self.input = None
        self.grad_weight = None
        self.grad_bias = None

    def forward(self, x):
        """
        Прямое распространение для линейного слоя
        
        Args:
            x: входной тензор формы (batch_size, input_size)
        
        Returns:
            выходной тензор формы (batch_size, output_size)
        """
        # TODO: Сохраните входные данные для backward pass
        self.input = x

        # TODO: Реализуйте линейное преобразование
        output = self.input @ self.weight

        if self.use_bias:
            output = output + self.bias

        return output

    def backward(self, grad_output):
        """
        Обратное распространение для линейного слоя
        
        Args:
            grad_output: градиент от следующего слоя формы (batch_size, output_size)
        
        Returns:
            градиент для предыдущего слоя формы (batch_size, input_size)
        """
        # TODO: Вычислите градиент по входу
        grad_input = grad_output @ self.weight.T

        # TODO: Вычислите градиент по весам
        self.grad_weight = self.input.T @ grad_output

        # TODO: Вычислите градиент по bias
        if self.use_bias:
            self.grad_bias = np.sum(grad_output, axis=0)
        return grad_input

    def update_weights(self, learning_rate=0.01):
        """
        Обновление весов с помощью градиентного спуска
        """
        if self.grad_weight is not None:
            self.weight -= learning_rate * self.grad_weight

        if self.use_bias and self.grad_bias is not None:
            self.bias -= learning_rate * self.grad_bias


In [408]:
class Sequential(Layer):
    def __init__(self, *layers):
        super().__init__()
        self.layers = list(layers)
        self.layer_outputs = []

    def add(self, layer):
        """
        Добавление слоя в последовательность
        """
        self.layers.append(layer)

    def forward(self, x):
        """
        Прямое распространение через все слои
        
        Args:
            x: входной тензор
        
        Returns:
            выходной тензор после прохождения всех слоев
        """
        # TODO: Очистите список промежуточных выходов
        self.layer_outputs = []

        # TODO: Последовательно примените все слои
        output = x
        for layer in self.layers:
            # TODO: Применить слой и сохранить результат
            output = layer.forward(output)  # Применяем слой
            self.layer_outputs.append(output)  # Сохраняем выход слоя
        return output

    def backward(self, grad_output):
        """
        Обратное распространение через все слои в обратном порядке
        
        Args:
            grad_output: градиент от следующего слоя
        
        Returns:
            градиент для предыдущего слоя
        """
        # TODO: Примените backward для всех слоев в обратном порядке
        grad = grad_output
        for layer in reversed(self.layers):
            # TODO: Примените backward для текущего слоя
            grad = layer.backward(grad)  # Применяем слой
        return grad

    def train(self):
        """
        Переключение всех слоев в режим обучения
        """
        super().train()
        for layer in self.layers:
            layer.train()

    def eval(self):
        """
        Переключение всех слоев в режим инференса
        """
        super().eval()
        for layer in self.layers:
            layer.eval()

    def __len__(self):
        return len(self.layers)

    def __getitem__(self, idx):
        return self.layers[idx]


In [409]:
class Dropout(Layer):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.mask = None

    def forward(self, x):
        """
        Прямое распространение для Dropout
        
        Args:
            x: входной тензор
        
        Returns:
            выходной тензор с примененным dropout (в режиме обучения)
        """
        if self.training:
            # TODO: Создайте бинарную маску для dropout
            self.mask = np.random.binomial(1, 1 - self.dropout_rate, size=x.shape)

            # TODO: Примените маску и масштабирование
            output = (x * self.mask) / (1 - self.dropout_rate)
        else:
            # TODO: В режиме инференса
            output = x
            self.mask = None

        return output

    def backward(self, grad_output):
        """
        Обратное распространение для Dropout
        
        Args:
            grad_output: градиент от следующего слоя
        
        Returns:
            градиент для предыдущего слоя
        """
        if self.training:
            # TODO: Примените ту же маску к градиенту
            grad_input = grad_output * self.mask
        else:
            grad_input = grad_output
        return grad_input


In [410]:
class BatchNorm(Layer):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        # TODO: Инициализируйте обучаемые параметры gamma и beta
        self.gamma = np.ones(num_features)
        self.beta = np.zeros(num_features)
        # TODO: Инициализируйте накопленную статистику
        self.running_mean = np.zeros(num_features)
        self.running_var = np.ones(num_features)

        # Переменные для backward pass
        self.batch_mean = None
        self.batch_var = None
        self.normalized = None
        self.input = None
        self.grad_gamma = None
        self.grad_beta = None

    def forward(self, x):
        """
        Прямое распространение для Batch Normalization
        
        Args:
            x: входной тензор формы (batch_size, num_features)
        
        Returns:
            нормализованный выходной тензор той же формы
        """
        self.input = x

        if self.training:
            # TODO: Вычислите статистику текущего batch
            self.batch_mean = np.mean(x, axis=0)
            self.batch_var = np.var(x, axis=0)

            # TODO: Обновите накопленную статистику
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * self.batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * self.batch_var

            mean = self.batch_mean
            var = self.batch_var
        else:
            # TODO: Используйте накопленную статистику
            mean = self.running_mean
            var = self.running_var

        # TODO: Нормализация
        self.normalized = (x - mean) / np.sqrt(var + self.eps)

        # TODO: Масштабирование и сдвиг
        output = self.gamma * self.normalized + self.beta

        return output

    def backward(self, grad_output):
        m = grad_output.shape[0]

        # --- градиенты по γ и β ---
        self.grad_gamma = np.sum(grad_output * self.normalized, axis=0)
        self.grad_beta = np.sum(grad_output, axis=0)

        # dL/dy * γ
        grad_y = grad_output * self.gamma

        # выбираем статистику (как в forward)
        if self.training:
            mean = self.batch_mean
            var = self.batch_var
        else:
            mean = self.running_mean
            var = self.running_var

        # dL/d(var)
        dvar = np.sum(
            grad_y * (self.input - mean) * -0.5 * (var + self.eps) ** (-3 / 2),
            axis=0
        )

        # dL/d(mean)
        dmean = np.sum(-grad_y / np.sqrt(var + self.eps), axis=0) + \
                dvar * np.mean(-2.0 * (self.input - mean), axis=0)

        # dL/dx
        grad_input = grad_y / np.sqrt(var + self.eps) + \
                     dvar * 2 * (self.input - mean) / m + \
                     dmean / m

        return grad_input

    def update_weights(self, learning_rate=0.01):
        """
        Обновление параметров
        """
        if self.grad_gamma is not None:
            self.gamma -= learning_rate * self.grad_gamma

        if self.grad_beta is not None:
            self.beta -= learning_rate * self.grad_beta


In [411]:
def softmax(x):
    """
    Устойчивая реализация softmax
    """
    # TODO: Реализуйте softmax функцию
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))

    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


def one_hot_encode(labels, num_classes):
    """
    Преобразование меток в one-hot кодировку
    """
    # TODO: Создайте one-hot кодировку
    return np.eye(num_classes)[labels]


class CrossEntropyLoss:
    def __init__(self):
        self.predictions = None
        self.targets = None

    def forward(self, predictions, targets):
        """
        Вычисление Cross-Entropy Loss

        Args:
            predictions: предсказания модели (batch_size, num_classes)
            targets: истинные метки класса (batch_size,)

        Returns:
            значение функции потерь
        """
        self.predictions = predictions
        self.targets = targets

        # TODO: Примените softmax к предсказаниям
        self.softmax_pred = softmax(self.predictions)

        # TODO: Вычислите cross-entropy loss
        loss = -np.mean(np.log(self.softmax_pred + 1e-8))

        return loss

    def backward(self):
        """
        Вычисление градиента Cross-Entropy Loss

        Returns:
            градиент по предсказаниям
        """
        # TODO: Вычислите градиент
        # Превращаем метки в one-hot (например, [2] → [0,0,1])
        targets_one_hot = np.eye(self.softmax_pred.shape[1])[self.targets]

        batch_size = self.predictions.shape[0]
        grad = (self.softmax_pred - targets_one_hot) / batch_size
        return grad


class MSELoss:
    def __init__(self):
        self.predictions = None
        self.targets = None

    def forward(self, predictions, targets):
        """
        Вычисление Mean Squared Error

        Args:
            predictions: предсказания модели
            targets: истинные значения

        Returns:
            значение функции потерь
        """
        self.predictions = predictions
        self.targets = targets
        batch_size = targets.shape[0]
        # TODO: Вычислите MSE
        loss = np.mean((predictions - targets) ** 2)
        return loss

    def backward(self, predictions, targets):
        batch_size = self.predictions.shape[0]
        grad = 2 * (self.predictions - self.targets) / batch_size
        return grad

In [412]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # TODO: Создайте архитектуру нейронной сети
        self.model = Sequential(
            Linear(input_size, hidden_size),
            # BatchNorm(hidden_size),
            Tanh(),
            # Dropout(0.8),
            Linear(hidden_size, output_size)
        )
        self.loss = MSELoss()
        self.learning_rate = 0.01
        self.epochs = 200


    def forward(self, x):
        return self.model.forward(x)


    def backward(self, grad_output):
        return self.model.backward(grad_output)


    def eval(self):
        self.model.eval()


    def get_trainable_layers(self):
        """
        Получение всех слоев с обучаемыми параметрами
        """
        trainable_layers = []
        for layer in self.model.layers:
            if hasattr(layer, 'update_weights'):
                trainable_layers.append(layer)
        return trainable_layers


    # MSE
    def get_loss(self, y_pred, y_true):
        return self.loss.forward(y_pred, y_true)


    def get_gradient(self, y_pred, y_true):
        return self.loss.backward(y_pred, y_true)


    def train(self, x_train, y_train, X_val=None, y_val=None):
        self.model.train()
        train_loses = []
        val_losses = []
        for epoch in range(self.epochs):
            y_pred = self.forward(x_train)
            loss = self.get_loss(y_pred, y_train)
            grad = self.get_gradient(y_pred, y_train)
            train_loses.append(loss)
            self.backward(grad)
            for layer in self.get_trainable_layers():
                layer.update_weights(self.learning_rate)
            if X_val is not None:
                self.eval()
                val_pred = self.forward(X_val)
                val_loss = self.get_loss(val_pred, y_val)
                val_losses.append(val_loss)
            log = f"Epoch {epoch + 1}/{self.epochs}, Train Loss: {loss:.4f}"
            if X_val is not None:
                log += f", Val Loss: {val_loss:.4f}"
            print(log)
        return train_loses, val_losses

# тест

In [413]:
n_samples = 200
n_features = 2
n_val = 50
np.random.seed(42)

X = np.random.randn(n_samples, n_features)

# Настоящая функция: y = x1*2 - x2*3 + шум
true_w = np.array([2.0, -3.0])
y = X @ true_w + np.random.randn(n_samples) * 0.5  # добавляем шум
y = y.reshape(-1, 1)  # делаем столбец (n_samples, 1)

# Разбиваем на train/val
X_train, y_train = X[:-n_val], y[:-n_val]
X_val, y_val = X[-n_val:], y[-n_val:]
first_neural = NeuralNetwork(2, 8, 1)
train_loses, val_loses = first_neural.train(X_train, y_train, X_val=X_val, y_val=y_val)


Epoch 1/200, Train Loss: 12.9504, Val Loss: 9.7478
Epoch 2/200, Train Loss: 12.0186, Val Loss: 9.0470
Epoch 3/200, Train Loss: 11.1600, Val Loss: 8.3966
Epoch 4/200, Train Loss: 10.3668, Val Loss: 7.7918
Epoch 5/200, Train Loss: 9.6329, Val Loss: 7.2291
Epoch 6/200, Train Loss: 8.9532, Val Loss: 6.7055
Epoch 7/200, Train Loss: 8.3236, Val Loss: 6.2185
Epoch 8/200, Train Loss: 7.7405, Val Loss: 5.7657
Epoch 9/200, Train Loss: 7.2005, Val Loss: 5.3451
Epoch 10/200, Train Loss: 6.7006, Val Loss: 4.9545
Epoch 11/200, Train Loss: 6.2378, Val Loss: 4.5921
Epoch 12/200, Train Loss: 5.8095, Val Loss: 4.2560
Epoch 13/200, Train Loss: 5.4131, Val Loss: 3.9446
Epoch 14/200, Train Loss: 5.0464, Val Loss: 3.6562
Epoch 15/200, Train Loss: 4.7072, Val Loss: 3.3893
Epoch 16/200, Train Loss: 4.3936, Val Loss: 3.1425
Epoch 17/200, Train Loss: 4.1038, Val Loss: 2.9146
Epoch 18/200, Train Loss: 3.8362, Val Loss: 2.7043
Epoch 19/200, Train Loss: 3.5893, Val Loss: 2.5104
Epoch 20/200, Train Loss: 3.3616, Va